In [333]:
import csv
import re
import pandas as pd
import random
import copy
import nltk

NAME = "fleep"
GENDER = "M"
CLASS = "hunter"
RACE = "dwarf"

quests = pd.read_csv("quests.csv")
#print(list(quests.columns.values))
details = quests.Details
rawStr = ""
for d in details:
    rawStr += str(d)

In [385]:
prepositions = ["were","of","the","from","is","on","when","at","for"] #words we don't want to end a sentence with
def get_ran_word(cfdist,word,interest):
    if interest in cfdist[word].keys() and random.random() > 0.1 and interest is not word:
        return interest
    if "." in cfdist[word].keys() and random.random() > 0.5 and word not in prepositions:
        return "."
    d = copy.copy(cfdist[word])
    lend = len(d)
    if lend == 0:
        return random.choice(["the","of"])
    if lend > 10:
        sampleNum = lend // 5; #sample from the top 20% words
    else:
        sampleNum = lend
    words = []
    for i in range(sampleNum):
        if(lend == 1):
            words.append(d.keys()[0])
            break;
        words.append(d.max());
        del(d[words[i]]);
    return random.choice(words)

def generate_ran_sentence(cfdist, word, interest="", minwords=5, maxwords = 15):#, num=15):
    i = 1
    initword = word
    words = [word.title()]
    word = get_ran_word(cfdist,word,interest)
    while word is not "." and i < maxwords:
        words.append(word)
        word = get_ran_word(cfdist,word,interest)
        i += 1
    if (interest is not "" and interest not in words) or len(words) < minwords: #keep going til you get an interest
        return generate_ran_sentence(cfdist,initword,interest=interest,minwords=minwords,maxwords=maxwords)
    return words
        
def format_simple(document):
    doc = document.lower()                                           #convert to lower case
    doc = re.sub(r'[\?\{\}\[\]\|\(\)!,:\-;\$\"<>]',' ',doc)           #remove punctuations
    doc = re.sub(r'[\']','',doc)                                     # remove appostrophes
    doc = re.sub(r'\.', ' . ', doc)                                  #add spaces between periods
    doc = re.sub(r'[\s]+',' ', doc)                                  #remove whitespace clumps
    doc =  doc.strip()                                               #remove trailing whitespace
    return doc

def replace_variables(document):
    doc = document.lower()
    if GENDER == "M":
        doc = re.sub(r'\$g(.*):(.*);',r'\g<1>',doc)
    else:
        doc = re.sub(r'\$g(.*):(.*);',r'\g<2>',doc)
    doc = re.sub(r'\$c',CLASS,doc)
    doc = re.sub(r'\$n',NAME,doc)
    doc = re.sub(r'\$r',RACE,doc)
    return doc
    
def format_simple_keep_punctuation(document):
    doc = document.lower()                                           #convert to lower case
    doc = re.sub(r'[\[\]\|<>\-]','',doc)                                  #remove unneeded punctuations
    doc = re.sub(r'([\.\?\{\}\[\]\(\)!,:\-;\$\"\'])',r' \g<1> ',doc)           #put spaces between tagged punctuations
    doc = re.sub(r'[\s]+',' ', doc)                                  #remove whitespace clumps
    doc =  doc.strip()                                               #remove trailing whitespace
    return doc

In [335]:
# rawStr = replace_variables(rawStr)
# rawStr = format_simple_keep_punctuation(rawStr)
# tokens = rawStr.split(' ')
# text = nltk.Text(tokens)

alldeets = "".join(str(d) for d in details)
alldeets = replace_variables(alldeets)
#construct text_nopunct
rawStr = format_simple(alldeets)
tokens_nopunct = rawStr.split(' ')
text_nopunct = nltk.Text(tokens_nopunct)
#construct text (with punctuation)
rawStr = format_simple_keep_punctuation(alldeets)
tokens = rawStr.split(' ')
text = nltk.Text(tokens_nopunct)

# get cfds from bigrams
bigrams_nopunct = nltk.bigrams(text_nopunct)
cfd_nopunct = nltk.ConditionalFreqDist(bigrams_nopunct)

bigrams = nltk.bigrams(text)
cfd = nltk.ConditionalFreqDist(bigrams)

In [304]:
postag = nltk.pos_tag(text)

In [305]:
tag_fd = nltk.FreqDist(tag for (word, tag) in postag)
print(tag_fd.most_common())
word_fd = nltk.FreqDist(word for (word, tag) in postag)
#print(word_fd.most_common())

[('NN', 100889), ('IN', 63764), ('DT', 57068), ('VB', 43443), ('.', 41763), ('JJ', 39215), ('PRP', 37256), ('NNS', 31305), ('RB', 26023), ('VBP', 21146), ('TO', 21129), ('CC', 15984), ('PRP$', 15421), ('VBZ', 13518), ('VBN', 12683), ('MD', 11906), ('VBD', 9451), ('VBG', 8848), ('RP', 3152), ('CD', 2746), ('WRB', 2373), ('WP', 2208), ('WDT', 1943), ('JJR', 1556), ('JJS', 1439), (':', 1180), ('EX', 1123), ('RBR', 645), ('PDT', 484), ('RBS', 322), ('FW', 109), ('UH', 95), ('NNP', 57), ('SYM', 55), ('WP$', 16), ('$', 3), ("''", 2), ('NNPS', 2)]


In [306]:
w = "hello"
print(cols)
print(cfd[w])
print(cfd[w].tabulate(10))
print(cfd[w].keys()[:10])
print(cfd[w].max())

yes


In [405]:
sentences = []
randsent1 = generate_ran_sentence(cfd_nopunct, "bold",interest="gold",maxwords=20)
randsent2 = generate_ran_sentence(cfd_nopunct, "old",interest="friend",maxwords=20)
sentences.append(" ".join(randsent1))
sentences.append(" ".join(randsent2))
print(". ".join(sentences) + ".")

Bold huh good gold coins ill. Old friend your friend whos been.


In [308]:
from nltk import PCFG
from nltk.parse.generate import generate, demo_grammar
from nltk import CFG
from operator import itemgetter
from random import shuffle

nonterms = {".",",",r"PRP$",":","$","\'\'", "(", ")", "CC", "CD", "DT", "EX", "FW", "IN", r"WP$"}
nonpos = {"i", "\"", "\'"}
termvalues = {}
#test = {"NN","JJ","JJR","MD","RB","VB","RBS","RBR","UH", "VBP", "VBZ", "WDT", "CC", "NNS", "PRP", "VBN", "EX"}

limit = 30
postags = postag
shuffle(postags)
postags.sort(key=itemgetter(1))
my_grammar = """
  S -> KI NK
  S -> COLCD N
  S -> TR NT
  S -> TK
  COLCD -> COL CD
  TK -> RT TO AND TAL TO NNP
  TK -> TR TO AND TAL TO NNP
  NK -> CD NN AT NNP
  NK -> NNP THE NNP
  NT -> TO NNP
  NT -> TO NNP AND KI CD NN
  NT -> TO NNP AND COL CD NN
  NT -> TO NNP AND KI NNP THE NNP
  N -> JJ NN FR NNP
  N -> JJ NN AT NNP
  N -> JJ NN
  N -> NN
  COL -> 'Collect' | 'Gather' | 'Find'
  FR -> 'from'
  KI -> 'Kill' | 'Slay'
  TR -> 'Travel' | 'Journey'
  AT -> 'At' | 'Located In'
  THE -> 'The'
  OF -> 'Of'
  AND -> 'And'
  RT -> 'Return'
  TAL -> 'Talk'
"""

my_grammar += "  " + "\n  ".join([(r"CD -> '" + str(num) + r"'") for num in [1,5,10]])
for (a,b) in postags:
    if b not in nonterms and a not in nonpos and len(a)>1:
        if termvalues.has_key(b) == False:
            termvalues[b] = 1
            my_grammar += "\n  " + "\n  ".join([(b + r" -> '" + a + r"'")])
        elif termvalues.get(b) < limit:
            termvalues[b] += 1
            my_grammar += "\n  " + "\n  ".join([(b + r" -> '" + a + r"'")]) 
        
#for (a,b) in postags:
#    if b not in nonterms and a not in nonpos:
#        if a == "you'll" and b not in test:
#            print(b +": " + a + " - " + b[0])

grammar = CFG.fromstring(my_grammar)
#print(grammar.productions())

#file = open('generation.txt', 'w')
#for sentence in generate(grammar):
    #file.write(' '.join(sentence))
#file.close()
fulllist = []
for sentence in generate(grammar):
    fulllist.append(sentence)

ranlist = [None]*10
for i in range(0,10):
    ranlist[i] = random.choice(fulllist)

for sentence in ranlist:
    print(' '.join(sentence))

#print(' '.join(random.choice(ranlist)))

#print(grammar)
#grammar += "\n\tN -> 'cat'"
#print(grammar)

#print(dir(CFG))
#print("\n")
#print(grammar.productions)


Travel to korkron And Slay youve The youve
Journey to xylem And Gather 10 vial
Travel to ymirheim And Kill ymirheim The moonwell
Find 10 enemy decision At zangarmarsh
Journey to ymirheim And Slay moonwell The ymirheim
Journey to zapnabber And Slay --is The --is
Journey to korkron And Slay mill The xs
Travel to xerash And Kill yer The kelris
Journey to moonwell And Kill -of The youve
Gather 10 enemy yeti from ymirheim
Travel to mill And Slay --was The --is
Travel to xerash And Slay ymirheim The &&&
Journey to xs And Kill xeleth The korkron
Travel to mighty And Slay --is The -c
Journey to kvaldir And Find 5 windseeker
Journey to kvaldir And Kill youve The zoram
Travel to kelris And Slay xylem The yer
Travel to moonwell And Kill 1 kelgash
Travel to --was And Collect 5 use
Journey to xiri And Kill mighty The zapnabber
Journey to xiri And Kill korkron The korkron
Journey to -c And Kill --was The ymirheim
Collect 1 enemy problem Located In moonwell
Travel to youve And Slay ymirheim The &&&
T